### This script transforms the measured and synthetic data into the formar required to perform the PV calculations

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from sklearn.metrics import mean_squared_error
import pickle
from datetime import timedelta as timedelta
from dateutil.relativedelta import relativedelta
from scipy.stats import *
# import datetime as dt

from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *
from Gauss_fit_functions import extractFIT, extractToPs , gauss, straight_line

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\os.py
c:\Users\Jason\thesis_project


# Load the measured data

In [2]:
X = genX([1994,2014], drop_0 = True)

dropping all zero rows


## Load IDs of profiles with the necesary amount of High and Low Season data

In [3]:
profileIDs = pd.read_pickle("ProfileIDs.pkl")

In [4]:
# # Collect only winter weekday profiles from dataset

# X.reset_index(inplace = True)

# df = X.copy()

# # Extract Season
# df['month'] = df.date.dt.month
# df['season'] = df['month'].apply(lambda x: 'winter' if x in [6, 7, 8] else 'summer') 
# df_summer = df[df['season'] == 'summer'] # Create dataframe with all the winter months, excluding weekends


# # Extract Weekdays
# weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
# df_summer['day_names'] = df_summer.date.dt.day_name()
# df_summer['daytype'] = df_summer.day_names.where(~df_summer.day_names.isin(weekdays), 'weekday')
# df_summer.drop(['day_names'], axis = 1, inplace = True)
# df_summer_weekdays  = df_summer[df_summer['daytype'] == 'weekday'] # Create dataframe with only weekdays
# df_summer_weekdays.drop(['month', 'season','daytype'], axis = 1, inplace = True)
# df_summer_weekdays.set_index(['ProfileID','date'], inplace=True)

# mean = df_summer_weekdays.groupby(["ProfileID"]).mean()

# # Test 1: Energy between 9-5 
# i = np.arange(9,17)
# sum_df = pd.DataFrame(index = mean.index, columns = ["9_to_5", 'total'])
# sum_df = sum_df.fillna(0)
# for x in i:
#     temp = mean.iloc[:,x]
#     sum_df['9_to_5'] = sum_df['9_to_5'] + temp

# sum_df['total'] =  mean.sum(axis = 1)

# sum_df["percentage"] = (sum_df["9_to_5"]/sum_df["total"]) * 100

# def check_9_to_5(df):
#     if df > 50:
#         return 'B'
#     else: 
#         return 'R'

# sum_df['Labels'] = sum_df['percentage'].apply(check_9_to_5)

# # 173 households labeled as Businesses for the 9-to-5 check
# sum_df[sum_df['percentage'] > 50]

# # Tests 2: ToP
# top_df = pd.DataFrame()

# top_df['Time'] = mean.idxmax(axis=1)

# def check_top(df):
#     if int(df) > 9 and int(df) < 17:
#         return 'B'
#     else:
#         return 'R'  

# top_df['Labels'] = top_df['Time'].apply(check_top)

# new_df = pd.DataFrame()

# new_df['ToP'], new_df['Sum'] = top_df['Labels'], sum_df['Labels']

# data_classes = ['R','B']
# d = dict(zip(data_classes, range(0,2)))

# new_df['ToP'],new_df['Sum'] = new_df['ToP'].map(d, na_action='ignore'),new_df['Sum'].map(d, na_action='ignore')

# new_df['Total'] = new_df.sum(axis = 1)

# residential_ids = new_df[new_df['Total'] == 0].index.values # Contains the profile ids that have typical residential behaviour

<ipython-input-4-64b8476c9274>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-64b8476c9274>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [414]:
# df_summer_weekdays.reset_index(inplace = True)

# # Remove all households with less than three winter months
# winter_months_ids = []

# for i in df_summer_weekdays['ProfileID'].unique():
#     if len(df_summer_weekdays[df_summer_weekdays['ProfileID'] == i]) >= 30:

#         winter_months_ids.append(i) # Contains the profile Ids that have a complete winter months data

# summer_months_ids = np.asarray(winter_months_ids)

In [415]:
# profileIDs = np.intersect1d(summer_months_ids, residential_ids)

# cleaned_df = df_summer_weekdays[df_summer_weekdays['ProfileID'].isin(profileIDs)]

# # # Load the previously identified profiles
# # id_identified_profiles = pd.read_pickle('sorted_households_winter_weekdays.pkl') # Read pickle file
# # res = []
# # for key in id_identified_profiles.keys():
# #     res.append(id_identified_profiles[key].tolist())

# # flat_list = list()
# # for sub_list in res:
# #     flat_list += sub_list

# # household_ids = flat_list

# # temp_df = cleaned_df.copy()
# # # temp_df.drop(['date'], axis = 1, inplace = True)
# # temp_df = temp_df.loc[temp_df.ProfileID.isin(household_ids),:]

# # cleaned_df = temp_df

# high_season_profileIDs = pd.read_pickle('ProfileIDs.pkl') # Read pickle file

# temp_ids = cleaned_df['ProfileID'].unique()

# profileIDs = np.intersect1d(high_season_profileIDs, temp_ids)

## Check that every household has one year of data
### if not take the first week of the months data as being representative of the months weekly usage and suplement the measured data

In [8]:
X.reset_index(inplace=True)

In [9]:
measured_DF = X.loc[X.ProfileID.isin(profileIDs),:]

### Create function that replaces missing dates

#### Attempt 1: Broken

In [419]:

# transformed_df = pd.DataFrame()

# for id in profileIDs:
#     data_new =  measured_DF[measured_DF['ProfileID'] == id].copy()

#     # Determine a households range of dates available
#     start_date = data_new['date'].iloc[0]
#     end_date = data_new['date'].iloc[-1]

#     date_range = pd.date_range(start = start_date, end = end_date)

#     # Get list of all the missing dates
#     missing_dates = date_range.difference(data_new['date'])

#     data_new.set_index(['date'], inplace = True)

#     for missing_date in missing_dates:
#         # Check if there is data measured one week prior
#         try:
#             # Get the date one week prior to missing date
#             previous_weeks_date = missing_date - timedelta(weeks=1)
#             data = data_new.loc[previous_weeks_date].tolist()
#             # print('Previous Weeks Date')
#         except KeyError:
#             following_weeks_date = missing_date + timedelta(weeks=1)
#             data = data_new.loc[following_weeks_date].tolist()
        

#         data_new.loc[missing_date] = data # Append list at the bottom
#         data_new = data_new.sort_index()#.reset_index(drop = False)
#         temp_df = data_new

#     transformed_df.append(temp_df)

In [231]:
# Check whether the day is high or low season
    # Check whether value is weekday or weekend
    # If date is weekday:
        # Try to get value from first of month same weekday
        # Else: Try to get any weekday value from month
        # Else: Get weekday value from a month within the same season
    # If date is weekend:
        # Repeat similiar process shown above
# df = data_new.copy()

# df.reset_index(inplace = True)

# df['month'] = df.date.dt.month
# df['season'] = df['month'].apply(lambda x: 'high_season' if x in [6, 7, 8] else 'low_season') 

# # Extract Weekdays
# weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
# weekend = ['Saturday','Sunday']
# df['day_names'] = df.date.dt.day_name()
# # df['day_names'] = df.day_names.where(~df.day_names.isin(weekdays), 'WeekDay')
# # df['day_names'] = df.day_names.where(~df.day_names.isin(weekend), 'WeekEnd') 


In [286]:
# missing_day = missing_date.day_name()
# missing_month = missing_date.month
# data = df[(df['month'] == missing_month) & (df['day_names'] == missing_day)]
# # df = df.set_index(['date'], inplace = True)
# df.loc[missing_date] = data.values[0] # Append list at the bottom
# df = df.sort_index()#.reset_index(drop = False)
# temp_df = df

## Function that replace missing data in measured data

In [11]:
temporary_df = pd.DataFrame(columns = measured_DF.columns)##measured_DF.set_index(['date'])
temporary_df.set_index(['date'], inplace = True)

High_season = [6,7,8]

for id in tqdm(profileIDs):
    data_new =  measured_DF[measured_DF['ProfileID'] == id].copy()

    data_new['month'] = data_new.date.dt.month
    data_new['season'] = data_new['month'].apply(lambda x: 'high_season' if x in [6, 7, 8] else 'low_season') 
    data_new['day_names'] = data_new.date.dt.day_name()

    # Extract Weekdays
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
    weekends = ['Sunday', 'Saturday']
    data_new['day_names'] = data_new.date.dt.day_name()
    data_new['daytype'] = data_new.day_names.apply(lambda x: 'Weekday' if x in weekdays else 'Weekend') 


    # Determine a households range of dates available
    start_date = data_new['date'].iloc[0]
    end_date = data_new['date'].iloc[-1]

    one_year_later = start_date + relativedelta(years = 1)

    if one_year_later < end_date:
        date_range = pd.date_range(start = start_date, end = end_date)
    else:
        date_range = pd.date_range(start = start_date, end = one_year_later)

    # Get list of all the missing dates
    missing_dates = date_range.difference(data_new['date'])

    data_new.set_index(['date'], inplace = True)

    for missing_date in missing_dates:
        missing_day = missing_date.day_name()
        missing_month = missing_date.month
        # Check if there is data measured one week prior

        # Get the date one week prior to missing date
        data = data_new[(data_new['month'] == missing_month) & (data_new['day_names'] == missing_day)]
        if len(data) != 0:
            data_new.loc[missing_date] = data.sample(n = 1).values[0] # Append list at the bottom
            data_new = data_new.sort_index()#.reset_index(drop = False)
            temp_df = data_new
        else:
            x = missing_date.month
            if x in High_season:
                data = data_new[(data_new['season'] == 'high_season') & (data_new['day_names'] == missing_day)]
                if len(data) != 0:
                    data_new.loc[missing_date] = data.sample(n = 1).values[0] # Append list at the bottom
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekdays:
                    data = data_new[(data_new['season'] == 'high_season') & (data_new['daytype'] == 'Weekday')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekends:
                    data = data_new[(data_new['season'] == 'high_season') & (data_new['daytype'] == 'Weekend')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
            else:
                data = data_new[(data_new['season'] == 'low_season') & (data_new['day_names'] == missing_day)]
                if len(data) != 0:
                    data_new.loc[missing_date] = data.sample(n = 1).values[0] # Append list at the bottom
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekdays:
                    data = data_new[(data_new['season'] == 'low_season') & (data_new['daytype'] == 'Weekday')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekends:
                    data = data_new[(data_new['season'] == 'low_season') & (data_new['daytype'] == 'Weekend')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new

        # print('Previous Weeks Date')

    # temp_df.drop(['month', 'season', 'day_names'], axis = 1, inplace = True)
    temporary_df = temporary_df.append(temp_df)
    

temporary_df.to_csv("Measured_Consumption_Missing_Days_filled.csv")



<ipython-input-11-8342425998b7>:6: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/9993 [00:00<?, ?it/s]

In [13]:
measured_profiles = temporary_df.drop(['month','season','day_names','daytype'], axis =1)

In [63]:
measured_profiles.to_csv('Measured_Profiles_missing_values_replaced.csv')

In [16]:
measured_profiles.reset_index(inplace = True)

# Check that every user has one year of data

### Change dates so that all users profiles starts in January and ends in December

In [136]:
def change_dates(measured_profiles_1):

    measured_profiles_1 = measured_profiles[measured_profiles['ProfileID'] == 1]
    start_date = measured_profiles_1['date'].iloc[0]
    one_year_later = start_date + relativedelta(years = 1)
    measured_profiles_1['date'] = measured_profiles_1['date'].apply(lambda x: x.replace(year = start_date.year) if x < one_year_later else x )
    measured_profiles_1['date'] = measured_profiles_1['date'].apply(lambda x: x if x.year == start_date.year else 'drop' )
    measured_profiles_1 = measured_profiles_1[measured_profiles_1['date'] != 'drop']
    measured_profiles_1 = measured_profiles_1.set_index(['date']).sort_index()

    return measured_profiles_1

### Adapt above code to run for all users

In [139]:
testing_df = measured_profiles.copy()
testing_df_2 = testing_df.groupby(['ProfileID']).apply(change_dates)

In [142]:
testing_df_2.to_csv('Measured_Profiles_Missing_days_replaced_sorted.csv')

# Create the synthetic years worth of data for each user

### Load the gauss fit features

In [186]:
FitFeatures_HWeekend = pd.read_csv("FitFeatures_High_Season_Weekends.csv",index_col=0)
FitFeatures_HWeekday = pd.read_csv("FitFeatures_HighSeason_weekdays.csv", index_col=0)
FitFeatures_LWeekend = pd.read_csv("FitFeatures_Low_Season_weekends.csv", index_col=0)
FitFeatures_LWeekday = pd.read_csv("FitFeatures_Low_Season_weekdays.csv", index_col=0)

### Load the ampltude distributions

In [171]:
Distributions_HWeekday = pd.read_csv('High_Season_Weekday_Distributions.csv', header = [0,1], index_col = 0)
Distributions_HWeekend = pd.read_csv('High_Season_Weekends_Distributions.csv', header = [0,1], index_col = 0)
Distributions_LWeekday = pd.read_csv('Low_Season_Weekday_Distributions.csv', header = [0,1], index_col = 0)
Distributions_LWeekend = pd.read_csv('Low_Season_Weekend_Distributions.csv', header = [0,1], index_col = 0)

### Load the peak amplitdues of every household

In [180]:
Amplitudes_HWeekday = pd.read_csv("High_Season_weekday_amplitudes.csv", index_col = 0)
Amplitudes_HWeekend = pd.read_csv("High_Season_weekends_amplitudes.csv", index_col = 0)
Amplitudes_LWeekday = pd.read_csv("Low_Season_weekdays_amplitudes.csv", index_col = 0)
Amplitudes_LWeekend = pd.read_csv("Low_Season_weekend_amplitudes.csv", index_col = 0)

### Get the standard deviation of the measured peaks

In [182]:
def standard_deviation(my_list):
    #calculate population standard deviation of list 
    return (sum((x-(sum(my_list) / len(my_list)))**2 for x in my_list) / len(my_list))**0.5

In [193]:
def get_amplitudes_std(temporary):

    std_deviation_df = pd.DataFrame(index = temporary.index.unique())

    for id in tqdm(temporary.index.unique()):
        try:
            std_deviation_df.loc[id,'A1_std'] = standard_deviation(temporary.loc[id]['A3'])
            std_deviation_df.loc[id,'A2_std'] = standard_deviation(temporary.loc[id]['A2'])
            std_deviation_df.loc[id,'mu1_std'] = standard_deviation(temporary.loc[id]['mu1'])
            std_deviation_df.loc[id,'mu2_std'] = standard_deviation(temporary.loc[id]['mu2'])
        except TypeError:
            print('TypeError')
            continue
        
    return std_deviation_df

In [194]:
STD_HWeekday = get_amplitudes_std(Amplitudes_HWeekday)
STD_HWeekend = get_amplitudes_std(Amplitudes_HWeekend)
STD_LWeekday = get_amplitudes_std(Amplitudes_LWeekday)
STD_LWeekend = get_amplitudes_std(Amplitudes_LWeekend)

<ipython-input-193-605ecf8491d3>:5: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/9869 [00:00<?, ?it/s]

  0%|          | 0/3549 [00:00<?, ?it/s]

<ipython-input-193-605ecf8491d3>:5: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/9111 [00:00<?, ?it/s]

TypeError


  0%|          | 0/9539 [00:00<?, ?it/s]

TypeError
TypeError
TypeError
TypeError


### Create the dataframe with all the fit features

In [196]:
Combined_HWeekday = pd.merge(FitFeatures_HWeekday,STD_HWeekday, left_index = True, right_index = True)
Combined_HWeekend = pd.merge(FitFeatures_HWeekend,STD_HWeekend, left_index = True, right_index = True)
Combined_LWeekday = pd.merge(FitFeatures_LWeekday,STD_LWeekday, left_index = True, right_index = True)
Combined_LWeekend = pd.merge(FitFeatures_LWeekend,STD_LWeekend, left_index = True, right_index = True)


In [198]:
Combined_HWeekday

,H_offset,sigma1,sigma2,mu1,A1,sigma3,sigma4,mu2,A2,A1_std,A2_std,mu1_std,mu2_std
ProfileID,,,,,,,,,,,,,
1,1.338480,1.538462,0.634921,7,1.839216,2.631579,1.034483,21,2.988971,1.524691,4.144478,1.572146,2.016839
3,7.292170,0.631579,1.081081,7,5.727844,1.428571,1.860465,20,8.090200,5.316935,5.734053,2.135647,2.568181
5,1.642967,1.071429,1.562500,7,12.090489,2.000000,1.739130,20,13.238194,3.734773,4.208459,1.066960,2.274852
6,1.416962,1.038961,0.960000,7,10.967371,1.739130,2.121212,17,5.962964,2.612709,4.590361,1.149413,1.614234
10,1.446746,1.100917,2.941176,6,3.465118,1.250000,2.051282,17,4.772254,1.586904,1.996307,1.533401,1.876822
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12029045,2.774317,1.750000,2.564103,8,19.158962,3.703704,2.800000,17,14.632787,5.093918,3.138364,0.992311,1.554008
12029049,0.229599,2.051282,0.563380,8,0.262805,2.040816,1.944444,17,1.096630,0.838400,1.103542,2.409524,1.253057
12029058,11.227044,1.149425,1.875000,5,11.530503,1.944444,2.181818,18,32.465252,4.310748,7.779670,1.977265,0.557163


In [159]:
profileIDs1 = np.intersect1d(FitFeatures_HWeekend.ProfileID.values, FitFeatures_HWeekday.ProfileID.values)
profileIDs2 = np.intersect1d(FitFeatures_LWeekend.ProfileID.values, FitFeatures_LWeekday.ProfileID.values)

profileIDs = np.intersect1d(profileIDs1,profileIDs2)

In [163]:
synthetic_profiles_1 = pd.DataFrame(columns = measured_profiles_1.columns,index = measured_profiles_1.index)

### Get the standard deviation of the amplitdues

In [237]:
def  generate_Synthetic_Profiles(houseID, distributions_df, number_of_days, combined_df):

    # distributions_df = Distributions_HWeekday
    size = number_of_days
    level_0 = ['A1','A2']
    
    inv_data_df = pd.DataFrame()
    temp_df_inv = pd.DataFrame()
    # for houseID in tqdm(distributions_df.index):
    for column in level_0:

        distributions = distributions_df[column].loc[houseID]['Distribution']
        parameters = distributions_df[column].loc[houseID]['params']
        print(parameters[1])
        loc = combined_df.loc[houseID][column] + combined_df.loc[houseID]['H_offset']
        scale = combined_df.loc[houseID][column + '_std']  
        # loc = parameters[-2]
        # scale = parameters[-1]
        print(f"loc = {loc}, scale = {scale}, parameters[0] = {parameters[0]}")

        try:
            if distributions == 'invgauss':
                print('invgauss')
                data_points = invgauss.rvs(parameters[0],loc = loc,scale = scale,size = size)
            elif distributions == 'weibull_min':
                print('weibull_min')
                data_points = weibull_min.rvs(parameters[0], loc = loc,scale = scale, size = size)
            elif distributions == 'weibull_max':
                print('weibull_max')
                print(f'loc = {loc}, scale = {scale}')
                data_points = weibull_max.rvs(parameters[0], loc = loc, scale = scale, size = size)
                # data_points = weibull_max.rvs(parameters[0], parameters[1], parameters[2], size = 60)
            elif distributions == 'beta':
                print('beta')
                data_points = beta.rvs(parameters[0], parameters[1], loc = loc,scale = scale, size = size)
                # data_points = beta.rvs(parameters[0], parameters[1], parameters[2],parameters[3], size = 60)
            elif distributions == 'norm':
                print('norm')
                data_points = norm.rvs(parameters[0], parameters[1], size = size)
            elif distributions == 'triang':
                print('triang')
                data_points = triang.rvs(parameters[0], loc = loc,scale = scale, size = size)
            elif distributions == 'pearson3':
                print('pearson3')
                data_points = pearson3.rvs(parameters[0], loc = loc,scale = scale, size = size)
            elif distributions == 'lognorm':
                print('lognorm')
                data_points = lognorm.rvs(parameters[0], loc = loc,scale = scale, size = size)
            elif distributions == 'uniform':
                print('uniform')
                data_points = uniform.rvs(loc = loc,scale = scale, size = size)
            elif distributions == 'expon':
                print('expon')
                data_points = expon.rvs(loc = loc,scale = scale, size = size)
            elif distributions == 'gamma':
                print('gamma')
                data_points = gamma.rvs(parameters[0], loc = loc,scale = scale, size = size)
            elif distributions == 'alpha':
                print('alpha')
                data_points = alpha.rvs(parameters[0], loc=loc, scale = scale,size=size)
            elif distributions == 'arcsine':
                print('arcsine')
                data_points = arcsine.rvs(loc=loc, scale = scale,size=size)
            elif distributions == 'cauchy':
                print('cauchy')
                data_points = cauchy.rvs(loc=loc, scale = scale,size=size)
            elif distributions == 'dweibull':
                print('dweibull')
                data_points = dweibull.rvs(parameters[0], loc=loc, scale = scale,size=size)
            elif distributions == 'exponnorm':
                print('exponnorm')
                data_points = exponnorm.rvs(K=1.5, loc=loc, scale = scale,size=size)
            elif distributions == 'halflogistic':
                print('halflogistic')
                data_points = halflogistic.rvs(loc=loc, scale = scale,size=size)
            elif distributions == 'logistic':
                print('logistic')
                data_points = logistic.rvs(loc=loc, scale = scale,size=size)
            elif distributions == 'loggamma':
                print('loggamma')
                data_points = loggamma.rvs(parameters[0], loc=loc, scale = scale,size=size)
            elif distributions == 'powerlaw':
                print('powerlaw')
                data_points = powerlaw.rvs(parameters[0], loc=loc, scale = scale,size=size)
            elif distributions == 'powernorm':
                print('powernorm')
                data_points = powernorm.rvs(parameters[0], loc=loc, scale = scale,size=size)
        except ValueError:
            print("ValueError")
            continue
        

    # print(column)
    # inverse_data_points = inverse_StandardScalar(data_points,temporary.loc[id],column,0.99,0.01)
    temp_df_inv['ProfileID'] = houseID
    temp_df_inv[column] = data_points#inverse_data_points
        
    inv_data_df = inv_data_df.append(temp_df_inv)
    temp_df_inv = pd.DataFrame()
        # if column == 'A1':
        #     data_pointA1 = data_points#inverse_data_points
        # elif column == 'A2':
        #     data_pointsA2 = data_points#inverse_data_points

    inv_data_df = inv_data_df.dropna()
    inv_data_df.set_index(['ProfileID'], inplace = False)

In [ ]:
distributions_df[column].loc[houseID]['params']

In [238]:
# generate_Synthetic_Profiles(houseID, distributions_df, number_of_days, combined_df):
# distributions_df = Distributions_HWeekday, Distributions_HWeekend, Distributions_LWeekday, Distributions_LWeekend
# number_of_days = 1
# combined_df = Combined_HWeekday, Combined_HWeekend, Combined_LWeekday, Combined_LWeekend
# distributions_id = Distributions_HWeekday['A1'].loc[houseID]
Weekdays = ['Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday']
Weekends = ['Saturday', 'Sunday']
HSeason = [6,7,8]
for id in profileIDs:
    temp_synth = pd.DataFrame(index = testing_df_2[testing_df_2['ProfileID'] == 1].index, columns = testing_df_2.columns)

    for date in temp_synth.index:
        
        if (date.month in HSeason) and (date.day_name() in Weekdays):
            distributions_df = Distributions_HWeekday
            combined_df = Combined_HWeekday
        elif (date.month in HSeason) and (date.day_name() not in Weekdays):
            distributions_df = Distributions_HWeekend
            combined_df = Combined_HWeekend
        elif (date.month not in HSeason) and (date.day_name() in Weekdays):
            distributions_df = Distributions_LWeekday
            combined_df = Combined_LWeekday
        elif (date.month not in HSeason) and (date.day_name() not in Weekdays):
            distributions_df = Distributions_LWeekend
            combined_df = Combined_LWeekend
    
        temp_synth.loc[date] = generate_Synthetic_Profiles(id, distributions_df, 1, combined_df)

0
loc = 1.8021396636493048, scale = 2.0833120338877404, parameters[0] = (
invgauss


TypeError: '>' not supported between instances of 'numpy.ndarray' and 'int'

In [197]:
measured_profiles_1
Weekdays = ['Monday', 'Tuesday','Wednesday','Thursday','Friday']
Weekends = ['Saturday','Sunday']


if date.day_name in 

,ProfileID,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
1994-01-01,1,1.800000,1.800000,1.850000,1.700000,1.850000,1.966667,1.200000,1.20,1.200000,...,1.20,1.150000,1.20,1.200000,1.20,1.150000,1.20,1.200000,1.150000,1.516667
1994-01-02,1,1.233333,1.400000,1.366667,1.400000,1.700000,1.433333,1.150000,1.20,1.200000,...,1.20,1.200000,1.15,1.150000,0.90,1.200000,1.20,1.150000,1.200000,1.233333
1994-01-03,1,1.366667,1.400000,1.650000,1.700000,1.533333,2.050000,1.150000,1.20,1.200000,...,1.20,1.200000,1.15,1.200000,0.95,1.200000,1.20,1.150000,1.200000,1.266667
1994-01-04,1,1.366667,1.400000,1.450000,1.750000,2.016667,2.533333,1.400000,1.20,1.200000,...,1.15,1.200000,1.20,1.050000,1.00,1.000000,1.20,1.150000,1.366667,1.650000
1994-01-05,1,2.250000,2.483333,2.300000,2.400000,2.366667,2.400000,2.000000,1.50,1.266667,...,1.20,1.150000,1.10,1.150000,1.00,1.200000,1.15,1.266667,1.500000,1.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994-12-27,1,1.200000,1.200000,1.150000,1.200000,1.200000,1.150000,1.200000,1.20,1.200000,...,1.20,1.066667,1.15,1.200000,1.05,1.050000,1.20,1.200000,1.200000,1.150000
1994-12-28,1,1.200000,1.200000,1.150000,1.150000,1.200000,1.466667,1.200000,1.15,1.200000,...,1.15,1.150000,1.00,0.900000,0.90,1.133333,1.10,1.200000,1.150000,1.200000
1994-12-29,1,1.200000,1.200000,1.150000,1.200000,1.200000,1.150000,1.200000,1.20,1.200000,...,1.20,1.150000,1.10,0.950000,1.00,1.000000,1.20,1.200000,1.200000,1.150000
